# Validation with sklearn

In [16]:
##%matplotlib inline
import csv
import nltk
import os
from collections import defaultdict
import multiprocessing
import utils
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from id_stopwords import stopwords
import codecs
from sklearn.cluster import DBSCAN
from itertools import groupby
from random import shuffle
from collections import namedtuple

import crawler.settings
from crawler import db
from crawler.db import Product

In [2]:
import sys

In [3]:
# adjust this path to point to 'site-aggregation'
sys.path.append(os.path.join(os.environ["HOME"], "site-aggregation"))

In [7]:
crawler.settings.MYSQL_PASSWORD = "123"

In [9]:
eps = 0.1
include_title = True
include_description = True

In [8]:
#list(Product.select(Product.title, Product.url).limit(10))
db.mysql_db.connect()

In [17]:
CsvRow = namedtuple('CsvRow', 'id, title, url')
def ilines(
        file_name=None, 
        include_description=include_description):
    """
    File line iterator
    """
    global categories, prod_ids, category_rows
    prod_ids = []
    category_rows = defaultdict(lambda: [])
    
    for i, row in enumerate(
            Product.select(
                Product.id,
                Product.title,
                Product.description,
                Product.url)
            .where(Product.url % "https://www.bukalapak.com/p/%") if file_name is None
            else (CsvRow(*l) for l in utils.unicode_csv_reader(file_name)) ):
        prod_ids.append(row.id)
        category = '/'.join(row.url.split('/')[4:-1])
        category_rows[category].append(i)
        res = row.title
        if file_name is None and include_description and row.description:
            res += (' ' + row.description)
        yield res

        categories = sorted(category_rows.keys())

    return

In [ ]:
## Testing:
db.mysql_db.connect()
for i, l in enumerate(ilines(
        #file_name="/home/nad2000/python-scrapy-data-mining/corpus.csv", 
        include_description=True)):
    print l
    if i > 10:
        break

In [18]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(
    #input="filename",
    #max_df=1.1,
    #max_features=200000,
    #min_df=0.01,
    stop_words=stopwords,
    use_idf=True,
    tokenizer=utils.tokenize_and_stem_ID,
    ngram_range=(1,1))

In [19]:
%%time
db.mysql_db.connect()
tfidf_matrix = tfidf_vectorizer.fit_transform(ilines())
print tfidf_matrix.shape

(250111, 153767)
CPU times: user 1min 42s, sys: 1.36 s, total: 1min 44s
Wall time: 1min 46s


In [ ]:
#with open("tokes.txt", "w") as of:
#    for l in ilines(os.path.join(utils.data_dir, "corpus.csv")):
#        print >>of, utils.tokenize_and_stem_ID(l.lower())

In [20]:
#sorted([(c, len(ids)) for c, ids in category_products.items()], key=lambda e: -e[1])[50:60]

In [21]:
def get_clusters(category, eps=eps):
    """
    Runs clustering and returns labeled non-distinct product entires
    grouped by lables in a dictionary.
    """
    global tfidf_matrix, category_rows, row_nos
    
    row_nos = category_rows[category]

    # create and fit the model:
    db = DBSCAN(eps=eps).fit(tfidf_matrix[row_nos])
    return [
        (label, [p for _, p in prodi]) for label, prodi in groupby(
                ((l, prod_ids[row_nos[i]]) for i, l in sorted(enumerate(db.labels_), key=lambda e: -e[1]) if l != -1),
                key=lambda e: e[0]
        )
    ]

In [22]:
def category_clusters(categories):
    return [(c, get_clusters(c)) for c in categories]

In [23]:
%%time
shuffle(categories)
pool = multiprocessing.Pool(processes=4)
clusters = pool.map(category_clusters, utils.chunks(categories, 4))

CPU times: user 116 ms, sys: 184 ms, total: 300 ms
Wall time: 1min 50s


In [24]:
clusters = filter(lambda c: c[1], sum(clusters, []))

In [25]:
for c, labels in clusters:
    for l, products in labels:
        print "\"{0}\", {1}: {2}".format(c, l, ','.join(map(str, products)))

"fashion/couple/jam-tangan-947", 1: 142085,142087,142296,142399,142400
"fashion/couple/jam-tangan-947", 0: 142084,142086,142090,142091,142092
"hobi/olahraga/jersey", 0: 201098,201099,201100,201102,201104,201105,201111,201122,201124
"rumah-tangga/home-stuff", 1: 53940,53947,54578,54584,137790
"rumah-tangga/home-stuff", 0: 26428,26429,26431,26432,26433,26435,26439,26450,65496
"handphone/aksesoris-handphone", 0: 16396,65555,85261,139378,154028
"fashion/wanita/tas-wanita", 7: 161495,161497,161502,161515,161520,161571,161572,161577
"fashion/wanita/tas-wanita", 6: 161332,161333,161336,161339,161340,161341,161343,161345,161347,161349,161354,161355,161356,161357,161358,161359,161360,161361,161362,161363,161364,161365,161366,161367,161368,161369,161370,161371,161372,161373,161374,161375,161376,161377,161378,161379,161383,161384,161385,161386,161387,161388,161389,161390,161393,161394,161395,161396,161397,161398,161399,161400,161401,161402,161403,161404,161405,161406,161407,161408,161409,161410,1